# 1. Data loading

In [ ]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np

filename = 'NorthSeaData/FORCE_2020_train.csv' # nome do dado de entrada
df = pd.read_csv(filename) # leitura do dado de entrada

In [ ]:
### number feature (well log profiling) visualization
df.columns

In [ ]:
# indata 
training_data = df[['WELL', 
                    'DEPTH_MD', 
                    'RMED', 
                    'RDEP', 
                    'RHOB', 
                    'GR', 
                    'NPHI',
                    'DTC', 
                    'PEF', 
                    'FORCE_2020_LITHOFACIES_LITHOLOGY']].copy()

In [ ]:
# indata to use 
df = df[['WELL', 
         'DEPTH_MD', 
         'RMED', 
         'RDEP', 
         'RHOB', 
         'GR', 
         'NPHI',
         'DTC', 
         'PEF', 
         'FORCE_2020_LITHOFACIES_LITHOLOGY']].copy()

In [ ]:
df.columns

In [ ]:
# List the number of wells
for well in df['WELL'].unique():
    
    print(well)

In [ ]:
# List of headers
plot_cols = ['WELL', 'DEPTH_MD','RMED', 'RDEP', 'RHOB', 'GR', 'NPHI',
             'DTC', 'PEF','FORCE_2020_LITHOFACIES_LITHOLOGY']

In [ ]:
df = df[plot_cols]

In [ ]:
df.head()

In [ ]:
data_nan = df.copy()

In [ ]:
for num, col in enumerate(data_nan.columns[2:]):
    data_nan[col] = data_nan[col].notnull() * (num + 1)
    data_nan[col].replace(0, num, inplace=True)
    print(col, num) #Print out the col name and number to verify it works

In [ ]:
data_nan.describe()

# 2. Plotting the Data with and without NaN

In [ ]:
grouped = data_nan.groupby('WELL')

In [ ]:
#Setup the labels we want to display on the x-axis
#labels = ['RMED','RDEP', 'RHOB','GR', 'NPHI', 'DTC'] # 6 features

#labels = ['CALI','RMED', 'RDEP', 'RHOB', 'GR', 'NPHI', 'PEF','DTC', 'SP','DTS','DRHO', 'RMIC','RXO'] # 13 features
labels = ['RMED', 'RDEP', 'RHOB', 'GR', 'NPHI','DTC', 'PEF']


#Setup the figure and the subplots
fig, axs = plt.subplots(3, 4, figsize=(20,10))

#Loop through each well and column in the grouped dataframe
for (name, df), ax in zip(grouped, axs.flat):
    #ax.set_xlim(0,5) # 6 features
    ax.set_xlim(0,6) # 9 features
    
    #Setup the depth range
    ax.set_ylim(4000, 0)
    
    #Create multiple fill betweens for each curve# This is between
    # the number representing null values and the number representing
    # actual values
    
    #ax.fill_betweenx(df.DEPTH_MD, 0, df.CALI, facecolor='grey')
    ax.fill_betweenx(df.DEPTH_MD, 0, df.RMED, facecolor='lightgrey')
    ax.fill_betweenx(df.DEPTH_MD, 1, df.RDEP, facecolor='mediumseagreen')
    ax.fill_betweenx(df.DEPTH_MD, 2, df.RHOB, facecolor='lightblue')
    ax.fill_betweenx(df.DEPTH_MD, 3, df.GR, facecolor='lightcoral')
    ax.fill_betweenx(df.DEPTH_MD, 4, df.NPHI, facecolor='violet')
    ax.fill_betweenx(df.DEPTH_MD, 5, df.DTC, facecolor='darksalmon')
    ax.fill_betweenx(df.DEPTH_MD, 6, df.PEF, facecolor='red')
    #ax.fill_betweenx(df.DEPTH_MD, 6, df.SP, facecolor='thistle')
  
    
    #Setup the grid, axis labels and ticks
    ax.grid(axis='x', alpha=0.5, color='black')
    ax.set_ylabel('DEPTH (m)', fontsize=14, fontweight='bold')
    
    #Position vertical lines at the boundaries between the bars
    ax.set_xticks([1,2,3,4,5,6,7], minor=False)
    
    #Position the curve names in the centre of each column
    ax.set_xticks([0.5,1.5,2.5,3.5,4.5,5.5,6.5], minor=True)
    
    #Setup the x-axis tick labels
    ax.set_xticklabels(labels,  rotation='vertical', minor=True, verticalalignment='bottom')
    ax.set_xticklabels('', minor=False)
    ax.tick_params(axis='x', which='minor', pad=-7)
    
    #Assign the well name as the title to each subplot
    ax.set_title(name, fontsize=16, fontweight='bold')

plt.savefig('missingdata_northsea.pdf')
plt.tight_layout()
plt.subplots_adjust(hspace=0.15, wspace=0.25)
plt.show()

# 3. Select the headers to use in the in-data

In [ ]:
training_data.rename(columns={'FORCE_2020_LITHOFACIES_LITHOLOGY':'FACIES'}, inplace=True)

In [ ]:
training_data

# 4. Column Remapping / Renaming

In [ ]:
lithology_numbers = {30000: 'Sandstone', # sandybrown
                     65030: 'Sandstone/Shale', #darkgoldenrod
                     65000: 'Shale', # olive
                     80000: 'Marl', #gainsboro
                     74000: 'Dolomite',
                     70000: 'Limestone',
                     70032: 'Chalk',
                     88000: 'Halite',
                     86000: 'Anhydrite',
                     99000: 'Tuff',
                     90000: 'Coal',
                     93000: 'Basement'}

second dictionary to tranform in integer

In [ ]:
simple_lithology_numbers = {30000: 1,
                            65030: 2,
                            65000: 3,
                            80000: 4,
                            74000: 5,
                            70000: 6,
                            70032: 7,
                            88000: 8,
                            86000: 9,
                            99000: 10,
                            90000: 11,
                            93000: 12}

In [ ]:
training_data['LITH'] = training_data['FACIES'].map(lithology_numbers)

In [ ]:
training_data['LITH_SI'] = training_data['FACIES'].map(simple_lithology_numbers)

# 5. View the number of samples of the whole data

In [ ]:
#plot the count of Facies
training_data['LITH_SI'].value_counts().sort_index().plot(kind='bar')
print(training_data['LITH_SI'].value_counts().sort_index())
X_ind = np.arange(0,11,1)
plt.title('Number of samples')
plt.xticks(X_ind,['Sandstone',
                  'Sandstone/Shale',
                  'Shale',
                  'Marl',
                  'Dolomite',
                  'Limestone',
                  'Chalk',
                  'Halite',
                  'Anhydrite',
                  'Tuff',
                  'Coal'])
plt.show()

# 6. Crossplot RHOB and NPHI (whole data)

In [ ]:
import seaborn as sns

g = sns.FacetGrid(training_data, col='LITH', col_wrap=4)
g.map(sns.scatterplot, 'NPHI', 'RHOB', alpha=0.5)
g.set(xlim=(-0.15, 1))
g.set(ylim=(3, 1))
plt.show()

In [ ]:
# remove NaN
training_data.dropna(inplace=True)

In [ ]:
for well in training_data['WELL'].unique():
    
    print(well)

# 7. sorting out the blind test well

In [ ]:
blind = training_data[training_data['WELL'] == '16/2-16'] #seleciona um poço apenas do dado
training_data = training_data[training_data['WELL'] != '16/2-16'] #remove o poço do dado
blind

In [ ]:
training_data['WELL'].unique()

In [ ]:
import seaborn as sns

g = sns.FacetGrid(training_data, col='LITH', col_wrap=4)
g.map(sns.scatterplot, 'NPHI', 'RHOB', alpha=0.5)
g.set(xlim=(-0.15, 1))
g.set(ylim=(3, 1))
plt.show()

In [ ]:
import seaborn as sns

g = sns.FacetGrid(blind, col='LITH', col_wrap=4)
g.map(sns.scatterplot, 'NPHI', 'RHOB', alpha=0.5)
g.set(xlim=(-0.15, 1))
g.set(ylim=(3, 1))
plt.show()

Two lithofacoes are exluded from data after dropping NaN.

In [ ]:
#plot the count of Facies
blind['LITH_SI'].value_counts().sort_index().plot(kind='bar')
print(blind['LITH_SI'].value_counts().sort_index())
X_ind = np.arange(0,7,1)
plt.title('Samples - Blind well')
plt.xticks(X_ind,['Sandstone',
                  'Sandstone/Shale',
                  'Shale',
                  'Marl',
                  'Limestone',
                  'Anhydrit',
                  'Tuff'])
plt.show()

In [ ]:
#['WELL', 'DEPTH_MD', 'RDEP', 'RHOB','GR', 'NPHI', 'PEF', 'DTC','SP']
#col_list = ['LITH_SI','RDEP', 'RHOB','GR', 'NPHI', 'PEF', 'DTC','SP']

col_list = ['LITH_SI','RMED', 'RDEP', 'RHOB', 'GR', 'NPHI','DTC', 'PEF']



plt.figure(figsize=(15,10))
i=0
for col in col_list:
    i+=1
    plt.subplot(3,4,i)
    plt.hist(training_data[col])
    plt.title(col)
plt.show()

In [ ]:
#plot the count of Facies
training_data['LITH_SI'].value_counts().sort_index().plot(kind='bar')
print(training_data['LITH_SI'].value_counts().sort_index())
X_ind = np.arange(0,11,1)
plt.title('Samples - Training wells')
plt.xticks(X_ind,['Sandstone',
                  'Sandstone/Shale',
                  'Shale',
                  'Marl',
                  'Dolomite',
                  'Limestone',
                  'Chalk',
                  'Halite',
                  'Anhydrite',
                  'Tuff',
                  'Coal'])
plt.show()

# 8. Prepare data for modeling and blind test well


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score

from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.metrics import classification_report

In [ ]:
features = ['RMED', 'RDEP', 'RHOB', 'GR', 'NPHI','DTC', 'PEF']



y = training_data['LITH_SI']
X = training_data[features]

In [ ]:
### Data for modelling

#scaler = StandardScaler().fit(X)
#X_stnd = scaler.transform(X)

# standarization of data for SVM
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
X.corr()

In [ ]:
### Blind test well

y_blind = blind['LITH_SI']
X_blind = blind[features]
X_blind_stnd = sc.transform(X_blind)

In [ ]:
#Plot loss and accuracy

import matplotlib.pyplot as plt
def plot_history(history):
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [1000$]')
    plt.plot(history.epoch, np.array(history.history['mae']), label='Train')
    plt.plot(history.epoch, np.array(history.history['val_mae']),label = 'Val')
    plt.legend()
    plt.ylim([0,max(history.history['val_mae'])])

def plot_prediction(test_labels, test_predictions):
    plt.figure()
    plt.scatter(test_labels, test_predictions)
    plt.xlabel('True Values [1000$]')
    plt.ylabel('Predictions [1000$]')
    plt.axis('equal')
    plt.xlim(plt.xlim())
    plt.ylim(plt.ylim())
    _ = plt.plot([-100, 100],[-100,100])

    plt.figure()
    error = test_predictions - test_labels
    plt.hist(error, bins = 50)
    plt.xlabel("Prediction Error [1000$]")
    _ = plt.ylabel("Count")

In [ ]:
def plot_confusion_matrix(cm,
                          classes,
                          normalize,
                          title='Confusion matrix',
                          cmap=plt.cm.Greys):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    

    if normalize:
        
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center", verticalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

# 9. Parameter optimization and classifier training

In [ ]:
# from sklearn.svm import SVC # To use Support Vector Machine
from sklearn import ensemble # To use Gradient Boosting and Random forest
# from sklearn.neighbors import KNeighborsClassifier # To use KNN
from sklearn.metrics import f1_score

In [ ]:
training_features = ['Ss','Ss/Sh','Sh','M','D','L','Ch','H','T','C']
# list_blind_full =   ['Ss','Ss/Sh','Sh','M','L','Ch','A','T','C']
list_blind_full =   ['Ss','Ss/Sh','Sh','M','L','A','T']
# why do not fit and transform  GRADIENT BOOST
X1_train, X1_test, y1_train, y1_test = train_test_split(X, y, test_size=0.4, random_state=42)

### 9.5 CNN

In [ ]:
import random
import numpy as np
import tensorflow as tf
random.seed(10)
np.random.seed(10)
tf.random.set_seed(10)
from tensorflow import keras
import pandas as pd


print(tf.__version__)

In [ ]:
print("GPU Available:", tf.config.list_physical_devices('GPU'))

In [ ]:
print(X_train.shape)
print(X_train[1].shape)
print(X_train[0])

In [ ]:
sample_size = X_train.shape[0] # number of samples in train set
time_steps  = X_train.shape[1] # number of features in train set
input_dimension = 1               # each feature is represented by 1 number

train_data_reshaped = X_train.reshape(sample_size,time_steps,input_dimension)
print("After reshape train data set shape:\n", train_data_reshaped.shape)
print("1 Sample shape:\n",train_data_reshaped[0].shape)
print("An example sample:\n", train_data_reshaped[0])

In [ ]:
test_data_reshaped = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [ ]:
test_data_reshaped.shape

In [ ]:
def build_conv1D_model():

    n_timesteps = train_data_reshaped.shape[1] #
    n_features  = train_data_reshaped.shape[2] # 
       
    
    model = keras.Sequential(name="model_conv1D")
    
    # 1st layer
    ks = 2
    model.add(keras.layers.Input(shape=(n_timesteps,n_features)))
    model.add(keras.layers.Conv1D(filters=200, kernel_size=ks, strides=1, padding='valid', activation='relu', name="Conv1D_1"))
    model.add(keras.layers.MaxPooling1D(pool_size=1))
    model.add(keras.layers.Conv1D(filters=200, kernel_size=ks, strides=1, padding='valid', activation='relu', name="Conv1D_2"))
    model.add(keras.layers.MaxPooling1D(pool_size=1))
    model.add(keras.layers.Conv1D(filters=200, kernel_size=ks, strides=1, padding='valid', activation='relu', name="Conv1D_3"))
    model.add(keras.layers.MaxPooling1D(pool_size=1))
    model.add(keras.layers.Conv1D(filters=200, kernel_size=ks, strides=1, padding='valid', activation='relu', name="Conv1D_4"))
    model.add(keras.layers.MaxPooling1D(pool_size=1))
    
    #model.add(keras.layers.MaxPooling1D(pool_size=1, name="MaxPooling1D_fisrt"))
    
    # Dense
    
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(50, activation='relu'))
    model.add(keras.layers.Dense(50, activation='relu'))
    model.add(keras.layers.Dense(50, activation='relu'))
    model.add(keras.layers.Dense(50, activation='relu'))
    model.add(keras.layers.Dense(12, activation='softmax'))


    optimizer_aux = tf.keras.optimizers.Adam()
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer_aux ,metrics = ['accuracy'])
    
    return model

model_conv1D = build_conv1D_model()
model_conv1D.summary()


In [ ]:
earlystoping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                patience=5,
                                                verbose=1,
                                                mode='auto',
                                                restore_best_weights=True)
checkpoint_filepath = 'weights.{epoch:02d}-{val_loss:.2f}.h5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                      save_weights_only=True,
                                                      monitor='val_accuracy',
                                                      mode='max',
                                                      verbose=1,
                                                      save_best_only=True)

In [ ]:
history_cnn = model_conv1D.fit(train_data_reshaped, y_train, validation_data = (test_data_reshaped,y_test),
                           batch_size = 512, 
                           callbacks = [model_checkpoint,earlystoping],
                           epochs = 1000,
                           verbose=1)

In [ ]:
plt.plot(history_cnn.history['loss'])
plt.plot(history_cnn.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])


In [ ]:
plt.plot(history_cnn.history['accuracy'])
plt.plot(history_cnn.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])

plt.show()

In [ ]:
pred_test_cnn = model_conv1D.predict(test_data_reshaped)

In [ ]:
pred_test_cnn = tf.argmax(pred_test_cnn, axis=1)

In [ ]:
test_loss, test_acc = model_conv1D.evaluate(test_data_reshaped,  y_test, verbose=2)

In [ ]:
print(classification_report(y_test, pred_test_cnn, target_names=training_features))
cm_test_cnn = confusion_matrix(y_test, pred_test_cnn)
plot_confusion_matrix(cm_test_cnn, training_features, normalize=True)

In [ ]:
microF1_test_cnn = f1_score(y_test, pred_test_cnn, average='micro')
print('Test Macro f1 score:', microF1_test_cnn)

In [ ]:
X_blind_reshaped = X_blind_stnd.reshape(X_blind_stnd.shape[0],X_blind_stnd.shape[1],1)
X_blind_reshaped.shape

In [ ]:
aux = model_conv1D.predict(X_blind_reshaped)

In [ ]:
pred_blind_cnn = tf.argmax(aux, axis=1)

In [ ]:
print(classification_report(y_blind, pred_blind_cnn))
cm_cnn = confusion_matrix(y_blind, pred_blind_cnn)
plot_confusion_matrix(cm_cnn, training_features, normalize=True)

In [ ]:
microF1_blind_cnn = f1_score(y_blind, pred_blind_cnn, average='micro')
print('Test Macro f1 score:', microF1_blind_cnn)

### 9.6 CNN (RBF)

In [ ]:
# import keras
# from keras.layers import Layer
# from keras import backend as K

# class RBFLayer(Layer):
#     def __init__(self, units, gamma, ** kwargs):
#         super(RBFLayer, self).__init__( ** kwargs)
#         self.units = units
#         self.gamma = K.cast_to_floatx(gamma)

#     def build(self, input_shape):
#         self.mu = self.add_weight(name = 'mu',
#                                   shape = (int(input_shape[1]), self.units),
#                                   initializer = 'uniform',
#                                   trainable = True)
#         super(RBFLayer, self).build(input_shape)

#     def call(self, inputs):
#         diff = K.expand_dims(inputs) - self.mu
#         l2 = K.sum(K.pow(diff, 2), axis = 1)
#         res = K.exp(-1 * self.gamma * l2)
#         return res
    
#     def compute_output_shape(self, input_shape):
#         return (input_shape[0], self.units)
from rbflayer import RBFLayer, InitCentersRandom

In [ ]:
def build_conv1D_rbf_model():
    #
    n_timesteps = train_data_reshaped.shape[1] #
    n_features  = train_data_reshaped.shape[2] # 
    #
    model_rbf = keras.Sequential(name="model_conv1D_rbf")
    # 1st layer
    ks = 2
    mp=1
    f=128
    model_rbf.add(keras.layers.Input(shape=(n_timesteps,n_features)))
    model_rbf.add(keras.layers.Conv1D(filters=f, kernel_size=ks, activation='relu', name="Conv1D_1"))
    model_rbf.add(keras.layers.Conv1D(filters=f, kernel_size=ks, activation='relu', name="Conv1D_2"))
    model_rbf.add(keras.layers.MaxPooling1D(pool_size=mp))
    model_rbf.add(keras.layers.Dropout(0.2))
    model_rbf.add(keras.layers.BatchNormalization())
    # # 2nd layer
    model_rbf.add(keras.layers.Input(shape=(n_timesteps,n_features)))
    model_rbf.add(keras.layers.Conv1D(filters=f, kernel_size=ks, activation='relu', name="Conv1D_3"))
    model_rbf.add(keras.layers.Conv1D(filters=f, kernel_size=ks, activation='relu', name="Conv1D_4"))
    model_rbf.add(keras.layers.MaxPooling1D(pool_size=mp))
    model_rbf.add(keras.layers.Dropout(0.2))
    model_rbf.add(keras.layers.BatchNormalization())
    # # 3rd layer
    model_rbf.add(keras.layers.Input(shape=(n_timesteps,n_features)))
    model_rbf.add(keras.layers.Conv1D(filters=f, kernel_size=ks, activation='relu', name="Conv1D_5"))
    model_rbf.add(keras.layers.Conv1D(filters=f, kernel_size=ks, activation='relu', name="Conv1D_6"))
    model_rbf.add(keras.layers.MaxPooling1D(pool_size=mp))
    model_rbf.add(keras.layers.Dropout(0.2))
    model_rbf.add(keras.layers.BatchNormalization())
    rbflayer = RBFLayer(f,betas=2.0,input_shape=(1,))
    model_rbf.add(rbflayer)
    # Dense
    model_rbf.add(keras.layers.Flatten())
    model_rbf.add(keras.layers.Dense(512, activation='relu'))
    model_rbf.add(keras.layers.Dropout(0.2))
    model_rbf.add(keras.layers.Dense(12, activation='softmax'))


    optimizer_aux = tf.keras.optimizers.Adam()
    model_rbf.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer_aux ,metrics = ['accuracy'])
    
    return model_rbf

model_conv1D_rbf = build_conv1D_rbf_model()
model_conv1D_rbf.summary()


In [ ]:
history_rbf = model_conv1D_rbf.fit(train_data_reshaped, y_train, validation_data = (test_data_reshaped,y_test),
                           batch_size = 512, 
                           callbacks = [model_checkpoint,earlystoping],
                           epochs = 1000,
                           verbose=1)
# history_rbf = model_conv1D_rbf.fit(
#     train_data_reshaped, 
#     y_train, 
#     epochs = 1000,
#     # steps_per_epoch=len(train_data_reshaped)/10,
#     validation_data = (test_data_reshaped,y_test),
#     validation_steps= len(test_data_reshaped),
#     batch_size = 512, 
#     callbacks = [model_checkpoint,earlystoping], 
    
#     verbose=1)

In [ ]:
plt.plot(history_rbf.history['loss'])
plt.plot(history_rbf.history['val_loss'])
plt.title('model loss CNN (RBF)')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])


In [ ]:
plt.plot(history_rbf.history['accuracy'])
plt.plot(history_rbf.history['val_accuracy'])
plt.title('model accuracy CNN (RBF)')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])

plt.show()

In [ ]:
pred_test_rbf = model_conv1D_rbf.predict(test_data_reshaped)

In [ ]:
pred_test_rbf = tf.argmax(pred_test_rbf, axis=1)

In [ ]:
print(classification_report(y_test, pred_test_rbf))
cm_test_rbf = confusion_matrix(y_test, pred_test_rbf)
plot_confusion_matrix(cm_test_rbf, training_features, normalize=True)

In [ ]:
microF1_test_rbf = f1_score(y_test, pred_test_rbf, average='micro')
print('Test Macro f1 score:', microF1_test_rbf)

In [ ]:
pred_blind_rbf = model_conv1D_rbf.predict(X_blind_reshaped)

In [ ]:
pred_blind_rbf = tf.argmax(pred_blind_rbf, axis=1)

In [ ]:
# list_blind = ['Ss',
#               'Ss/Sh',
#               'Sh',
#               'M',
#               'L',
#               'T']

print(classification_report(y_blind, pred_blind_rbf))
cm_rbf = confusion_matrix(y_blind, pred_blind_rbf)
plot_confusion_matrix(cm_rbf, training_features, normalize=True)

# 10. Model performance evaluation

I will use the diagnosis of confusion matrix from train data set to evaluate the model performance. The diagnosis of confusion matrix points how much percentage of the stone is correctly predicted.

In [ ]:
### To create a data frame recording the correct prediction (normalized) of 
### facies for each machine learning algorithm

mod_test_list = ['CNN','CNN-RBF']
cm_test_list = [cm_test_cnn, cm_test_rbf]
face_test_list = training_features
pred_test_df = pd.DataFrame(index=training_features, columns=mod_test_list)

for mod in mod_test_list:
    
    col_index = int(mod_test_list.index(mod))
    cm = cm_test_list[col_index]
    
    for face in face_test_list:
        row_index = training_features.index(face)
        #print(face, row_index, col_index)
        pred_test_df.iloc[row_index, col_index] = cm[row_index][row_index]/sum(cm[row_index])
        

### add the accuracy factor
df_1 = pd.DataFrame([[microF1_test_cnn, 
                      microF1_test_rbf]], index=['Accuracy'], columns=mod_test_list)    


pred_test_conc = pd.concat([pred_test_df,df_1])
pred_test_conc

In [ ]:
X_ind = np.arange(pred_test_df.shape[0])
(pred_df_index_list) = training_features
aux=0.1
plt.figure(figsize=(10,5))
plt.bar(X_ind, pred_test_df['CNN'], color='blue', width=aux)
plt.bar(X_ind+0.1, pred_test_df['CNN-RBF'], color='red', width=aux)

plt.xticks(X_ind, pred_df_index_list)
plt.xlabel('Facies')
plt.ylabel('Correct predictions')
plt.legend(labels=mod_test_list)
plt.savefig('canada_performance_evaluation_test_data.pdf',bbox_inches='tight')
plt.show()

# 11. Calssifier evluation using blind test well

I will use the same method shown in item4 for evaluation.

In [ ]:
### To create a data frame recording the correct prediction (normalized) of facies of blind test well for each machine learning algorithm

blind_class  = ['Sandstone',
                  'Sandstone/Shale',
                  'Shale',
                  'Marl',
                  'Limestone',
                  'Chalk',
                  'Anhydrite',
                  'Tuff']

mod_list = ['CNN','CNN-RBF']
cm_list = [cm_cnn, cm_rbf]
pred_df = pd.DataFrame(index=list_blind_full, columns=mod_list)

for mod in mod_list:
    col_index = int(mod_list.index(mod))
    cm = cm_list[col_index]
    
    for face in list_blind_full:
        
        row_index = list_blind_full.index(face)
        #print(face, row_index, col_index)
        pred_df.iloc[row_index, col_index] = cm[row_index][row_index]/sum(cm[row_index])



In [ ]:
blind

In [ ]:
X_ind = np.arange(pred_df.shape[0])

aux=0.1
plt.figure(figsize=(10,5))
plt.bar(X_ind, pred_df['CNN'], color='blue', width=aux)
plt.bar(X_ind+0.1, pred_df['CNN-RBF'], color='red', width=aux)
plt.xticks(X_ind, list_blind_full)
plt.xlabel('Facies')
plt.ylabel('Correct predictions')
plt.legend(labels=mod_list)
plt.savefig('canada_performance_evaluation_blind_data.pdf',bbox_inches='tight')
plt.show()

# 12. Plot the predicted facies for comparison**

In [ ]:
blind = blind.copy()
blind['CNN'] = pred_blind_cnn
blind['RBF'] = pred_blind_rbf


blind.head()

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

facies_colors = ['bisque',
                 'darkorange',
                 'darkgoldenrod',
                 'peachpuff',
                 'beige',
                 'white',
                 'red']

blind_class  = ['Ss',
                  'Ss/Sh',
                  'Sh',
                  'M',
                  'L',
                  'A',
                  'T']

def compare_facies_plot(logs, compare1, compare2, facies_colors):
      #make sure logs are sorted by depth
    logs = logs.sort_values(by='DEPTH_MD')
    cmap_facies = colors.ListedColormap(
            facies_colors[0:len(facies_colors)], 'indexed')
    num_colors = 7
    ztop=logs.DEPTH_MD.min(); zbot=logs.DEPTH_MD.max()
    
    cluster0 = np.repeat(np.expand_dims(logs['LITH_SI'].values,1), 100, 1)
    cluster1 = np.repeat(np.expand_dims(logs[compare1].values,1), 100, 1)
    cluster2 = np.repeat(np.expand_dims(logs[compare2].values,1), 100, 1)
    # cluster3 = np.repeat(np.expand_dims(logs[compare3].values,1), 100, 1)
    # cluster4 = np.repeat(np.expand_dims(logs[compare4].values,1), 100, 1)
    # cluster5 = np.repeat(np.expand_dims(logs[compare5].values,1), 100, 1)
    # cluster6 = np.repeat(np.expand_dims(logs[compare6].values,1), 100, 1)
    # cluster7 = np.repeat(np.expand_dims(logs[compare7].values,1), 100, 1)
    
    
    f, ax = plt.subplots(nrows=1, ncols=10, figsize=(18, 15))
    ax[0].plot(logs.RMED, logs.DEPTH_MD, '-',color='red')
    ax[1].plot(logs.RDEP, logs.DEPTH_MD, '-',color='blue')
    ax[2].plot(logs.RHOB, logs.DEPTH_MD, '-', color='red')
    ax[3].plot(logs.GR, logs.DEPTH_MD, '-', color='green')
    ax[4].plot(logs.NPHI, logs.DEPTH_MD, '--', color='blue')
    ax[5].plot(logs.DTC, logs.DEPTH_MD, '-', color='black')
    ax[6].plot(logs.PEF, logs.DEPTH_MD, '-', color='black')
    im0 = ax[7].imshow(cluster0, interpolation='none', aspect='auto',
                    cmap=cmap_facies,vmin=1,vmax=num_colors)
    im1 = ax[8].imshow(cluster1, interpolation='none', aspect='auto',
                    cmap=cmap_facies,vmin=1,vmax=num_colors)
    im2 = ax[9].imshow(cluster2, interpolation='none', aspect='auto',
                    cmap=cmap_facies,vmin=1,vmax=num_colors)
    # im3 = ax[10].imshow(cluster3, interpolation='none', aspect='auto',
    #                 cmap=cmap_facies,vmin=1,vmax=num_colors)
    # im4 = ax[11].imshow(cluster4, interpolation='none', aspect='auto',
    #                 cmap=cmap_facies,vmin=1,vmax=num_colors)
    # im4 = ax[12].imshow(cluster5, interpolation='none', aspect='auto',
    #                 cmap=cmap_facies,vmin=1,vmax=num_colors)
    # im4 = ax[13].imshow(cluster6, interpolation='none', aspect='auto',
    #                 cmap=cmap_facies,vmin=1,vmax=num_colors)
    # im4 = ax[14].imshow(cluster7, interpolation='none', aspect='auto',
    #                 cmap=cmap_facies,vmin=1,vmax=num_colors)
    
            
    divider = make_axes_locatable(ax[9])
    cax = divider.append_axes("right", size="20%", pad=0.05)
    cbar=plt.colorbar(im2, cax=cax)
    cbar.set_label((30*' ').join(blind_class))
    cbar.set_ticks(range(0,1)); cbar.set_ticklabels('')
    
    for i in range(len(ax)-3):
        ax[i].set_ylim(ztop,zbot)
        ax[i].invert_yaxis()
        ax[i].grid()
        ax[i].locator_params(axis='x', nbins=4)
    
    ax[0].set_xlabel("RMED")
    ax[0].set_xlim(0.2,5)
    
    ax[1].set_xlabel("RDEP")
    ax[1].set_xlim(0.2,5)
    
    ax[2].set_xlabel("RHOB")
    ax[2].set_xlim(1.95,2.95)
    
    ax[3].set_xlabel("GR")
    ax[3].set_xlim(0,150)
    
    ax[4].set_xlabel("NPHI")
    ax[4].set_xlim(0.45,-0.15)
    
    ax[5].set_xlabel("DTC")
    ax[5].set_xlim(logs.DTC.min(),logs.DTC.max())
    
    ax[6].set_xlabel("PEF")
    ax[6].set_xlim(logs.PEF.min(),logs.PEF.max())
    
    ax[7].set_xlabel('Facies')
    ax[8].set_xlabel(compare1)
    ax[9].set_xlabel(compare2)
    # ax[10].set_xlabel(compare3)
    # ax[11].set_xlabel(compare4)
    # ax[12].set_xlabel(compare5)
    # ax[13].set_xlabel(compare6)
    # ax[14].set_xlabel(compare7)
    
    ax[1].set_yticklabels([]); ax[2].set_yticklabels([]); ax[3].set_yticklabels([])
    ax[4].set_yticklabels([]); ax[5].set_yticklabels([]); ax[6].set_yticklabels([])
    ax[7].set_yticklabels([]); ax[8].set_yticklabels([]); ax[9].set_yticklabels([])
    # ax[10].set_yticklabels([]); ax[11].set_yticklabels([]); ax[12].set_yticklabels([])
    # ax[13].set_yticklabels([]); ax[14].set_yticklabels([])
    
    
    ax[5].set_xticklabels([])
    ax[6].set_xticklabels([])
    ax[7].set_xticklabels([])
    ax[8].set_xticklabels([])
    ax[9].set_xticklabels([])
    # ax[10].set_xticklabels([])
    # ax[11].set_xticklabels([])
    # ax[12].set_xticklabels([])
    # ax[13].set_xticklabels([])
    # ax[14].set_xticklabels([])
    f.suptitle('Well: %s'%logs.iloc[0]['WELL'], fontsize=14,y=0.94)

In [ ]:
compare_facies_plot(blind, 'CNN','RBF', facies_colors)